In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install plotly
%pip install numpy
%pip install matplotlib
%pip install nbformat
%pip install opencv-python
%pip install seaborn
%pip install scikit-image
%pip install keras
%pip install tensorflow
%pip install wandb
%pip install tqdm
%pip install tensorflow_hub




In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sn
import keras.backend as K
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

import wandb
wandb.login()
%env "WANDB_NOTEBOOK_NAME" "model_unbalanced_v2.ipynb"

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


import cv2
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import itertools

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    name="testrun unbalanced fer dataset",


    # track hyperparameters and run metadata with wandb.config
    config={
        "optimizer": "RMSprop(learning_rate=1e-4)",
        "loss": "binary_crossentropy",
        "metric": ["accuracy"],
        "epoch": 40
    }
)

In [ ]:
TRAIN_DIR = ('../../../data/raw/train')
TEST_DIR = ('../../../data/raw/test')

In [ ]:
def load_data(dir_path, img_size):
    X = []
    y = []
    labels = []
    for label_idx, label_name in enumerate(sorted(os.listdir(dir_path))):
        if not label_name.startswith('.'):
            label_path = os.path.join(dir_path, label_name)
            for file_name in sorted(os.listdir(label_path)):
                if not file_name.startswith('.'):
                    img_path = os.path.join(label_path, file_name)
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA).astype('float32') / 255
                    X.append(img)
                    y.append(label_idx)
                    labels.append(label_name)
    X = np.array(X)
    y = np.array(y)
    labels = np.array(labels)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

In [ ]:
IMG_SIZE= (48, 48)

In [ ]:
X_train, y_train, train_labels = load_data(TRAIN_DIR, IMG_SIZE)

In [ ]:
X_test, y_test, test_labels = load_data(TEST_DIR,IMG_SIZE)

In [ ]:
from keras.utils import to_categorical

y_train -= 1
Y_train = to_categorical(y_train, num_classes = 6)
Y_train.shape

In [ ]:
y_test -= 1
Y_test = to_categorical(y_test, num_classes=6)
Y_test.shape

In [ ]:
from keras.applications.vgg16 import VGG16

base_model = VGG16(
        weights=None,
        include_top=False, 
        input_shape=IMG_SIZE + (3,)
    )

base_model.summary()

In [ ]:
NUM_CLASSES = 6

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [ ]:
def deep_model(model, X_train, Y_train, epochs, batch_size):
   
    model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=['accuracy'])
    
    history = model.fit(X_train
                       , Y_train
                       , epochs=epochs
                       , batch_size=batch_size
                       , verbose=1
                       ,callbacks=[WandbMetricsLogger(log_freq=5)])
    return history

In [ ]:
epochs = 50
batch_size = 32

history = deep_model(model, X_train, Y_train, epochs, batch_size)

wandb.finish()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    cm = np.round(cm,2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
# Validate on test set

predictions = model.predict(X_test)
y_pred = [np.argmax(probas) for probas in predictions]


accuracy = accuracy_score(y_test, y_pred)
print('Test Accuracy = %.2f' % accuracy)

confusion_mtx = confusion_matrix(y_test, y_pred) 
test_labels_dict = {i: label for i, label in enumerate(test_labels)}
cm = plot_confusion_matrix(confusion_mtx, classes=list(test_labels_dict.items()), normalize=False)